In [ ]:
"""
==============================================================================
  Copyright (c) 2025, Tarbiat Modares University (TMU)
  All rights reserved.

  This code is provided "as is" without any warranties, express or implied,
  including but not limited to the warranties of merchantability and fitness
  for a particular purpose.

  Description:
  This Python code is developed for predicting the solubility of anti-cancer
  drugs in supercritical CO₂ using various modeling and computational techniques.

  Author: Tara Sabooni
  Contact:
    tarasabooni99@email.com (Tara Sabooni)
    fakhroleslam@modares.ac.ir (Mohammad Fakhroleslam)
    pourya.poursalehi@gmail.com (Pourya Poursalehi)

  Last update: 2025-04-13
==============================================================================
"""


ModuleNotFoundError: No module named 'keras'

In [ ]:
#Finding the best parameters for the GRU model
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import Adam
from google.colab import drive

#For running on google drive
# Mount Google Drive
#drive.mount('/content/drive')

# Load data
df = pd.read_csv('/content/drive/My Drive/drugdata.csv')
df = df.iloc[:, :6]
df = df.dropna()
X = df.iloc[:, :5]
y = df.iloc[:, 5]
d = 1000000

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

# Reshape the data to fit into the GRU input format (samples, time steps, features)
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Function to calculate evaluation metrics including RAE
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    rrmse = rmse / (np.max(y_true) - np.min(y_true))
    r2 = r2_score(y_true, y_pred)
    aapre = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # RAE calculation
    rae = np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true - np.mean(y_true)))

    return mae, mse, rmse, rrmse, r2, aapre, rae

# Define the hyperparameter search space
gru_units = list(range(1, 1000))  # GRU units

# Variable to track the best model
best_params = None
best_test_r2 = -np.inf
results = {}

# Exhaustive search over GRU units
model_id = 0
for gru_unit in gru_units:
    model_id += 1

    # Build the GRU model with specified GRU units
    model = Sequential()
    model.add(GRU(units=gru_unit, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
    model.add(Dense(10, activation='relu'))  # Example dense layer with fixed 10 units
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    # Train the model
    model.fit(X_train_scaled, y_train_scaled, epochs=1, batch_size=32, verbose=0, validation_split=0.15)

    # Make predictions
    y_train_pred_scaled = model.predict(X_train_scaled).flatten()
    y_test_pred_scaled = model.predict(X_test_scaled).flatten()

    # Denormalize the predictions
    y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).flatten()
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()

    # Calculate metrics for training and testing sets
    train_mae, train_mse, train_rmse, train_rrmse, train_r2, train_aapre, train_rae = calculate_metrics(y_train/d, y_train_pred/d)
    test_mae, test_mse, test_rmse, test_rrmse, test_r2, test_aapre, test_rae = calculate_metrics(y_test/d, y_test_pred/d)

    # Save results
    results[model_id] = {
        "gru_units": gru_unit,
        "train_mae": train_mae, "train_mse": train_mse, "train_rmse": train_rmse, "train_rrmse": train_rrmse,
        "train_r2": train_r2, "train_aapre": train_aapre, "train_rae": train_rae,
        "test_mae": test_mae, "test_mse": test_mse, "test_rmse": test_rmse, "test_rrmse": test_rrmse,
        "test_r2": test_r2, "test_aapre": test_aapre, "test_rae": test_rae
    }

    # Update the best configuration
    if test_r2 > best_test_r2:
        best_test_r2 = test_r2
        best_params = gru_unit

# Print the best model configuration and performance metrics
print(f"Best GRU units: {best_params}")
print(f"Best Test R²: {best_test_r2}")

# Print results for all models
for model_id, result in results.items():
    print(f"Model {model_id}: GRU units={result['gru_units']}")
    print(f"Train R²: {result['train_r2']:.4f}, Test R²: {result['test_r2']:.4f}")
    print(f"Test MAE: {result['test_mae']:.4f}, Test RMSE: {result['test_rmse']:.4f}, Test R²: {result['test_r2']:.4f}\n")


In [ ]:
#Performing cross-validation
# Import necessary libraries
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import Adam

# Load data
df = pd.read_csv('/content/drive/My Drive/drugdata.csv')
X = df.iloc[:, :5]
y = df.iloc[:, 5]
d = 1000000

# Initialize KFold with 4 splits
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# Store results for each fold and GRU configuration
fold_results = []
best_test_r2 = -np.inf
best_units = 0

# Trying different numbers of GRU units
units = [10, 20, 50, 100]
# Function to calculate evaluation metrics including RAE
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    rrmse = rmse / (np.max(y_true) - np.min(y_true))
    r2 = r2_score(y_true, y_pred)
    aapre = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # RAE calculation
    rae = np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true - np.mean(y_true)))

    return mae, mse, rmse, rrmse, r2, aapre, rae

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    print(f"Processing Fold {fold}...")

    # Split the data for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Scale the data
    scaler_X = MinMaxScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)

    scaler_y = MinMaxScaler()
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

    # Reshape the data to fit into GRU input format (samples, time steps, features)
    X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

    for unit in units:
        # Build the GRU model with two additional Dense layers
        model = Sequential()
        model.add(GRU(units=unit, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer=Adam(), loss='mse')

        # Train the model
        model.fit(X_train_scaled, y_train_scaled, epochs=1000, batch_size=32, verbose=0, validation_split=0.15)

        # Make predictions
        y_train_pred_scaled = model.predict(X_train_scaled).flatten()
        y_test_pred_scaled = model.predict(X_test_scaled).flatten()

        # Denormalize the predictions
        y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).flatten()
        y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()

        # Calculate metrics for training and testing sets
        train_mae, train_mse, train_rmse, train_rrmse, train_r2, train_aapre, train_rae = calculate_metrics(y_train/d, y_train_pred/d)
        test_mae, test_mse, test_rmse, test_rrmse, test_r2, test_aapre, test_rae = calculate_metrics(y_test/d, y_test_pred/d)

        # Save results for this fold and configuration
        fold_results.append({
            "fold": fold,
            "units": unit,
            "train_mae": train_mae, "train_mse": train_mse, "train_rmse": train_rmse, "train_rrmse": train_rrmse,
            "train_r2": train_r2, "train_aapre": train_aapre, "train_rae": train_rae,
            "test_mae": test_mae, "test_mse": test_mse, "test_rmse": test_rmse, "test_rrmse": test_rrmse,
            "test_r2": test_r2, "test_aapre": test_aapre, "test_rae": test_rae
        })

        # Update the best unit configuration based on R² on the test set
        if test_r2 > best_test_r2:
            best_test_r2 = test_r2
            best_units = unit

# Print results for each fold and GRU configuration
for result in fold_results:
    print(f"Fold {result['fold']}, GRU Units: {result['units']}")
    print(f"Train MAE: {result['train_mae']}, Test MAE: {result['test_mae']}")
    print(f"Train MSE: {result['train_mse']}, Test MSE: {result['test_mse']}")
    print(f"Train RMSE: {result['train_rmse']:.4f}, Test RMSE: {result['test_rmse']:.4f}")
    print(f"Train RRMSE: {result['train_rrmse']:.4f}, Test RRMSE: {result['test_rrmse']:.4f}")
    print(f"Train R²: {result['train_r2']:.4f}, Test R²: {result['test_r2']:.4f}")
    print(f"Train RAE: {result['train_rae']:.4f}, Test RAE: {result['test_rae']:.4f}\n")

print(f"Best GRU units configuration based on test R²: {best_units}")

# Plot the test R² for each GRU configuration
plt.figure(figsize=(8, 6))
for unit in units:
    test_r2_values = [result["test_r2"] for result in fold_results if result["units"] == unit]
    plt.plot(range(1, 5), test_r2_values, marker='o', linestyle='-', label=f'Units: {unit}')

plt.xlabel('Fold')
plt.ylabel('Test R²')
plt.title('Test R² vs Fold for Different GRU Units')
plt.legend()
plt.grid(True)
plt.show()
